In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

# Arguments

In [2]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [3]:
pretrained = './log/pggan/128x128/b0/checkpoint.pth.tar'
resume = ''

In [4]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 5000
start_epoch = 0
train_batch = 32
test_batch = 300
lr = 0.01
schedule = [1500, 3000]
momentum = 0
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/to_style1/l2sp' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
alpha = 0.1
beta = 0.1
fc_name = 'fc.'

best_acc = 0

In [5]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [6]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
source_model = EfficientNet.from_name(model_name, num_classes=num_classes)
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    source_model.load_state_dict(torch.load(pretrained)['state_dict'])
    model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128x128/b0/checkpoint.pth.tar'


In [9]:
source_model.to('cuda')
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.01M


In [10]:
for param in source_model.parameters():
    param.requires_grad = False
source_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [11]:
source_model_weights = {}
for name, param in source_model.named_parameters():
    source_model_weights[name] = param.detach()

# Model tuning

In [12]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - source_model_weights[name]) ** 2
    return sp_loss

In [13]:
summary(model, input_size=(3,128,128),device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 129, 129]               0
Conv2dStaticSamePadding-2           [-1, 32, 64, 64]             864
         GroupNorm-3           [-1, 32, 64, 64]              64
MemoryEfficientSwish-4           [-1, 32, 64, 64]               0
         ZeroPad2d-5           [-1, 32, 66, 66]               0
Conv2dStaticSamePadding-6           [-1, 32, 64, 64]             288
         GroupNorm-7           [-1, 32, 64, 64]              64
MemoryEfficientSwish-8           [-1, 32, 64, 64]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 32, 1, 1]             288
         I

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [16]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss = loss_main + alpha*loss_sp + beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss = loss_main + alpha*loss_sp + beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        test_loss, test_acc = test(val_target_loader, model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, model, criterion, epoch, use_cuda)

        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        scheduler_warmup.step()

        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)


Epoch: [1 | 5000] LR: 0.010000
1/7 | Total:0:00:01 | ETA:0:00:10 | Loss:3.6185648441314697 | top1:43.75
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:1.7886848747730255 | top1:44.79166793823242


/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


26/26 | Total:0:00:14 | ETA:0:00:00 | Loss:0.9832128355136285 | top1:49.858970642089844
107/107 | Total:0:00:17 | ETA:0:00:00 | Loss:0.02289659267935519 | top1:99.75701141357422

Epoch: [2 | 5000] LR: 0.017000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.8644558787345886 | top1:53.125
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.8808906078338623 | top1:51.5625

Epoch: [3 | 5000] LR: 0.017000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.7491140365600586 | top1:46.875
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.7803894480069479 | top1:51.5625

Epoch: [4 | 5000] LR: 0.017000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.6400483846664429 | top1:65.625
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.7663237353165945 | top1:51.04166793823242

Epoch: [5 | 5000] LR: 0.017000
1/7 | Total:0:00:01 | ETA:0:00:07 | Loss:0.6401665210723877 | top1:65.625
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.686393270889918 | top1:55.208335876464844

Epoch: [6 | 5000] LR: 0.017000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.598

1/7 | Total:0:00:01 | ETA:0:00:07 | Loss:0.2930944263935089 | top1:81.25
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.3128287394841512 | top1:85.9375

Epoch: [46 | 5000] LR: 0.017000
1/7 | Total:0:00:01 | ETA:0:00:07 | Loss:0.2819564640522003 | top1:87.5
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.2764525016148885 | top1:86.97917175292969

Epoch: [47 | 5000] LR: 0.017000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.3317457437515259 | top1:87.5
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.29100532581408817 | top1:88.02083587646484

Epoch: [48 | 5000] LR: 0.017000
1/7 | Total:0:00:01 | ETA:0:00:07 | Loss:0.22090475261211395 | top1:93.75
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.25899890313545865 | top1:88.54167175292969

Epoch: [49 | 5000] LR: 0.017000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.21065595746040344 | top1:93.75
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.20587781568368277 | top1:92.70833587646484

Epoch: [50 | 5000] LR: 0.017000
1/7 | Total:0:00:01 | ETA:0:00:08 | Loss:0.283894956

1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.10710124671459198 | top1:96.875
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.1272163155178229 | top1:95.83333587646484

Epoch: [89 | 5000] LR: 0.024000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.09618830680847168 | top1:96.875
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.1746167497088512 | top1:92.1875

Epoch: [90 | 5000] LR: 0.024000
1/7 | Total:0:00:01 | ETA:0:00:09 | Loss:0.12595367431640625 | top1:96.875
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.17524034281571707 | top1:92.1875

Epoch: [91 | 5000] LR: 0.024000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.058746207505464554 | top1:100.0
7/7 | Total:0:00:01 | ETA:0:00:01 | Loss:0.14065238771339258 | top1:95.83333587646484

Epoch: [92 | 5000] LR: 0.024000
1/7 | Total:0:00:01 | ETA:0:00:07 | Loss:0.05155321583151817 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.14364429004490376 | top1:94.79167175292969

Epoch: [93 | 5000] LR: 0.024000
1/7 | Total:0:00:01 | ETA:0:00:08 | Loss:0.1423370540

...

1/7 | Total:0:00:01 | ETA:0:00:08 | Loss:0.06335725635290146 | top1:96.875
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.02622205950319767 | top1:98.95833587646484

Epoch: [1400 | 5000] LR: 0.079998
1/7 | Total:0:00:01 | ETA:0:00:09 | Loss:0.0034859180450439453 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0029945572217305503 | top1:100.0

Epoch: [1401 | 5000] LR: 0.079998
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.001851361244916916 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.009519298871358236 | top1:99.47917175292969
26/26 | Total:0:00:20 | ETA:0:00:00 | Loss:2.0459480102245626 | top1:55.28205108642578
107/107 | Total:0:00:26 | ETA:0:00:00 | Loss:2.0148913425819894 | top1:54.65420913696289

Epoch: [1402 | 5000] LR: 0.079997
1/7 | Total:0:00:01 | ETA:0:00:11 | Loss:0.002828530967235565 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.005095801626642545 | top1:100.0

Epoch: [1403 | 5000] LR: 0.079997
1/7 | Total:0:00:01 | ETA:0:00:10 | Loss:0.0012004524469375

1/7 | Total:0:00:01 | ETA:0:00:09 | Loss:0.0009312629699707031 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0008603986352682114 | top1:100.0

Epoch: [1442 | 5000] LR: 0.079997
1/7 | Total:0:00:01 | ETA:0:00:09 | Loss:0.0012844577431678772 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.005214902261892955 | top1:100.0

Epoch: [1443 | 5000] LR: 0.079997
1/7 | Total:0:00:01 | ETA:0:00:10 | Loss:0.0004595443606376648 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0006733344246943792 | top1:100.0

Epoch: [1444 | 5000] LR: 0.079997
1/7 | Total:0:00:01 | ETA:0:00:07 | Loss:0.004020798951387405 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.016100460042556126 | top1:98.95833587646484

Epoch: [1445 | 5000] LR: 0.079997
1/7 | Total:0:00:01 | ETA:0:00:09 | Loss:0.0016568787395954132 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.002748271139959494 | top1:100.0

Epoch: [1446 | 5000] LR: 0.079997
1/7 | Total:0:00:01 | ETA:0:00:08 | Loss:0.00085820257663

7/7 | Total:0:00:03 | ETA:0:00:01 | Loss:0.0005179935445388159 | top1:100.0

Epoch: [1484 | 5000] LR: 0.079997
1/7 | Total:0:00:01 | ETA:0:00:07 | Loss:0.0022071823477745056 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.019068927504122257 | top1:99.47917175292969

Epoch: [1485 | 5000] LR: 0.079997
1/7 | Total:0:00:01 | ETA:0:00:09 | Loss:0.0008381158113479614 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.010141605511307716 | top1:99.47917175292969

Epoch: [1486 | 5000] LR: 0.079997
1/7 | Total:0:00:01 | ETA:0:00:08 | Loss:0.0030144229531288147 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.034729486952225365 | top1:98.95833587646484

Epoch: [1487 | 5000] LR: 0.079997
1/7 | Total:0:00:01 | ETA:0:00:10 | Loss:0.00037566572427749634 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.017603841920693714 | top1:99.47917175292969

Epoch: [1488 | 5000] LR: 0.079997
1/7 | Total:0:00:01 | ETA:0:00:07 | Loss:0.0013197064399719238 | top1:100.0
7/7 | Total:0:00:02 

7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.06989706059296925 | top1:97.39583587646484

Epoch: [1526 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:09 | Loss:0.0030197016894817352 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0013924278318881989 | top1:100.0

Epoch: [1527 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:08 | Loss:0.03652661293745041 | top1:96.875
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.03307581382493178 | top1:98.95833587646484

Epoch: [1528 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.0009171068668365479 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.012655346343914667 | top1:99.47917175292969

Epoch: [1529 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:07 | Loss:0.00032057613134384155 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0010534549752871196 | top1:100.0

Epoch: [1530 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:07 | Loss:0.001110270619392395 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 |

7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0020695744703213372 | top1:100.0

Epoch: [1568 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.0007221773266792297 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.014282970999677977 | top1:99.47917175292969

Epoch: [1569 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:05 | Loss:0.0006774887442588806 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.002490915668507417 | top1:100.0

Epoch: [1570 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:08 | Loss:0.0008177682757377625 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.002079185719291369 | top1:100.0

Epoch: [1571 | 5000] LR: 0.000000
1/7 | Total:0:00:01 | ETA:0:00:10 | Loss:0.003887306898832321 | top1:100.0
7/7 | Total:0:00:03 | ETA:0:00:01 | Loss:0.00244836633404096 | top1:100.0

Epoch: [1572 | 5000] LR: 0.000000
1/7 | Total:0:00:00 | ETA:0:00:06 | Loss:0.0032317861914634705 | top1:100.0
7/7 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0063327824076016